In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation

Using TensorFlow backend.


In [131]:
## Help Python find our packages
import sys
sys.path.append('..')

import json
import numpy as np
import matplotlib.pyplot as plt
import domain_scoring.domain_scoring as domain_scoring

# Randomness
import random as rn
import tensorflow as tf
import os

In [3]:
path = '../rated_datasets/Rotten_Tomato_Merlin_1519148528.2417703.json'

In [30]:
merlin = json.load(open(path, "r", encoding="utf8"))

In [5]:
path = '../rated_datasets/Rotten Tomato_Potato_1519142479.127663.json'

In [6]:
potato = json.load(open(path, "r", encoding="utf8"))

In [86]:
def extract_mps(data):
    count = 0
    first = True
    batches = []
    batch = []
    for probably_path in data["meta_paths"]:
        if count % 6 == 0:
            # Don't add empty batches
            if len(batch) > 0:
                batches.append(batch)
                batch = []
        else:
            if 'time_to_rate' not in probably_path.keys():
                batch.append(probably_path)
        count += 1
    # append last batch
    if len(batch) > 0:
        batches.append(batch)
    print('#meta-paths:', count - len(batches) - 1)
    return batches

In [87]:
def construct_graph(batches):
    ## Construct rating graph
    from util.datastructures import MetaPathRatingGraph, MetaPath
    graph = MetaPathRatingGraph()

    for batch in batches:
        #ordered = sorted(batch, key=lambda x: float(x['rating']))
        for metapath in batch:
            for another_metapath in batch:
                if metapath is another_metapath:
                    continue
                if float(metapath['rating']) <= float(another_metapath['rating']):
                    graph.add_user_rating(MetaPath.from_list(another_metapath['metapath']), MetaPath.from_list(metapath['metapath']), 
                                          distance=float(another_metapath['rating']) - float(metapath['rating']))
    return graph

In [88]:
## Clean up data // remove time_to_rate from array of ratings.
def clean_up(data):
    batches = extract_mps(data)
    return batches, construct_graph(batches)

In [89]:
p_batches, p_graph = clean_up(potato)
m_batches, m_graph = clean_up(merlin)

#meta-paths: 51
#meta-paths: 51


In [92]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [93]:
domain_score = domain_scoring.DomainScoring()

In [94]:
# Preprocess meta-paths
domain_score.vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), token_pattern='\\b\\w+\\b')

In [95]:
domain_score.fit(m_graph)

In [96]:
x, y = domain_score._extract_data_labels(m_graph)
x = domain_score._preprocess(x)

In [97]:
test_size = 0.3
random_state = 42

In [98]:
x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                    test_size=test_size,
                                                    random_state=random_state,
                                                    shuffle=True)

In [99]:
len(x_train)

131

In [105]:
np.array(x_train).shape

(131, 44)

In [155]:
# Reproducible results
np.random.seed(random_state)
rn.seed(random_state)
tf.set_random_seed(random_state)
os.environ['PYTHONHASHSEED'] = '0'

# Build model
model = Sequential()
model.add(Dense(44, activation='relu', input_dim=44))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train model
model.fit(np.array(x_train), np.array(y_train), epochs=10, batch_size=15)

Epoch 1/10
131/131 [==============================] - 0s 3ms/step - loss: 0.6947 - acc: 0.5191
Epoch 2/10
131/131 [==============================] - 0s 172us/step - loss: 0.6815 - acc: 0.5878
Epoch 3/10
131/131 [==============================] - 0s 237us/step - loss: 0.6762 - acc: 0.5573
Epoch 4/10
131/131 [==============================] - 0s 220us/step - loss: 0.6721 - acc: 0.6260
Epoch 5/10
131/131 [==============================] - 0s 183us/step - loss: 0.6684 - acc: 0.6641
Epoch 6/10
131/131 [==============================] - 0s 196us/step - loss: 0.6656 - acc: 0.6870
Epoch 7/10
131/131 [==============================] - 0s 131us/step - loss: 0.6614 - acc: 0.6870
Epoch 8/10
131/131 [==============================] - 0s 193us/step - loss: 0.6601 - acc: 0.6794
Epoch 9/10
131/131 [==============================] - 0s 240us/step - loss: 0.6551 - acc: 0.6870
Epoch 10/10
131/131 [==============================] - 0s 170us/step - loss: 0.6537 - acc: 0.6641


In [156]:
model.evaluate(np.array(x_test), np.array(y_test))

57/57 [==============================] - 0s 3ms/step


[0.6610821872426752, 0.6666666698037532]